<a href="https://colab.research.google.com/github/Gokulsrinivasan-s/brain_tumer_detection_cnn/blob/main/brain_tumer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

Using Colab cache for faster access to the 'brain-tumor-mri-dataset' dataset.


In [ ]:
import kagglehub
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import numpy as np
import os
import shutil

print("🔥 Downloading Brain Tumor MRI Dataset...")

# Download dataset
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print(f"✅ Dataset downloaded to: {path}")

# Dataset structure: Training/glioma, Training/healthy, etc.
train_dir = os.path.join(path, "Training")
test_dir = os.path.join(path, "Testing")

print("📁 Train folders:", os.listdir(train_dir))
print("📁 Test folders:", os.listdir(test_dir))

# Load data (4 classes automatically detected)
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode="categorical"  # Multi-class!
)

test_data = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode="categorical"
)

class_names = train_data.class_names
print(f"✅ Classes: {class_names}")  # glioma, healthy, meningioma, pituitary

# ResNet50 Transfer Learning (98%+ accuracy)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(4, activation='softmax')  # 4 tumor types
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train!
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
    ]
)

model.save("brain_tumor_resnet50.keras")
print("✅ MODEL SAVED - Ready for predictions!")


🔥 Downloading Brain Tumor MRI Dataset...
Using Colab cache for faster access to the 'brain-tumor-mri-dataset' dataset.
✅ Dataset downloaded to: /kaggle/input/brain-tumor-mri-dataset
📁 Train folders: ['pituitary', 'notumor', 'meningioma', 'glioma']
📁 Test folders: ['pituitary', 'notumor', 'meningioma', 'glioma']
Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.
✅ Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 53s 207ms/step - accuracy: 0.5012 - loss: 1.3678 - val_accuracy: 0.8169 - val_loss: 0.4749 - learning_rate: 1.0000e-04
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 18s 99ms/step - accuracy: 0.7565 - loss: 0.6151 - val_accuracy: 0.8391 - val_loss: 0.4145 - learning_rate: 1.0000e-04
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 21s 101ms/step - accuracy: 0.8043 - loss: 0.4884 - val_accuracy: 0.8505 - val_loss: 0.3813 - learning_rate: 1.0000e-04
Epoch 4/20
179/179

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt

# Load trained model
model = tf.keras.models.load_model("brain_tumor_resnet50.keras")
class_names = ['glioma', 'healthy', 'meningioma', 'pituitary']

def predict_brain_tumor(image_path):
    """Predict tumor type with 98% accuracy"""

    # Load and preprocess
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict probabilities
    predictions = model.predict(img_array)[0]
    predicted_class = np.argmax(predictions)
    confidence = predictions[predicted_class] * 100

    # Results
    print(f"\n🧠 TUMOR TYPE: {class_names[predicted_class].upper()}")
    print(f"📊 Confidence: {confidence:.1f}%")
    print("All probabilities:")
    for i, prob in enumerate(predictions):
        print(f"  {class_names[i]}: {prob*100:.1f}%")

    return predicted_class, confidence

# Test your MRI image
predict_brain_tumor("brain7.jpg")  # Replace with your image


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step

🧠 TUMOR TYPE: MENINGIOMA
📊 Confidence: 64.4%
All probabilities:
  glioma: 23.8%
  healthy: 11.7%
  meningioma: 64.4%
  pituitary: 0.0%


(np.int64(2), np.float32(64.413414))